## 🧪 White-box Testing: Branch Coverage and Cross-Platform Consistency

### 1️⃣ Background and Objectives
> This section aims to validate whether the `pickle` module produces consistent serialized outputs (via hash comparison) across different operating systems using branch-based white-box testing.


### 2️⃣ Environment Information
> Listing the platforms and Python versions used for testing.

In [38]:
import platform
import sys

def print_environment_info():
    print("📌 Current operating system:", platform.system(), platform.release())
    print("📌 Python version:", sys.version)

In [2]:
import platform
import os

def get_platform_key():
    """Get the current platform identifier（macOS / Windows / Linux）"""
    sysname = platform.system().lower()
    if 'darwin' in sysname:
        return 'macOS'
    elif 'windows' in sysname:
        return 'Windows'
    elif 'linux' in sysname:
        return 'Linux'
    return 'Unknown'

key = get_platform_key()


In [ ]:
# mac 
if key == 'macOS':
    print_environment_info()

📌 Current operating system: Darwin 24.1.0
📌 Python version: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 10:07:17) [Clang 14.0.6 ]


In [42]:
# windows 
if key == 'Windows':
    print_environment_info()

📌 Current operating system: Windows 11
📌 Python version: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)]


In [ ]:
# Liunx 
if key == 'Linux':
    print_environment_info()

📌 Current operating system: Linux 6.14.6-zen1-1-zen
📌 Python version: 3.12.4 (main, May 20 2025, 15:56:44) [GCC 15.1.1 20250425]


### 3️⃣ Test Cases and Input Structures
> Each input structure is mapped to a specific branch logic (e.g., empty vs. non-empty lists).

In [44]:
import pickle
import hashlib

def get_hash(obj):
    """Return the SHA256 hash value of the object after pickle serialization"""
    return hashlib.sha256(pickle.dumps(obj)).hexdigest()

In [45]:
def save_result(case_name, value_hash, file_path="branch_hashes.txt"):
    current_platform = get_platform_key()
    formatted_block = f"{case_name}\n"

    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
    else:
        content = ""

    blocks = content.strip().split("\n\n") if content else []
    updated = False

    for i in range(len(blocks)):
        if blocks[i].startswith(case_name):
            lines = blocks[i].split("\n")
            lines = [line for line in lines if not line.startswith(current_platform)]
            lines.append(f"{current_platform.ljust(10)}Result: {value_hash}")
            blocks[i] = "\n".join(lines)
            updated = True
            break

    if not updated:
        formatted_block += f"{current_platform.ljust(10)}Result: {value_hash}\n"
        if current_platform != "Windows":
            formatted_block += f"{'Windows'.ljust(10)}Result: Pending\n"
        if current_platform != "Linux":
            formatted_block += f"{'Linux'.ljust(10)}Result: Pending\n"
        if current_platform != "macOS":
            formatted_block += f"{'macOS'.ljust(10)}Result: Pending\n"
        blocks.append(formatted_block.strip())

    with open(file_path, "w", encoding="utf-8") as f:
        f.write("\n\n".join(blocks) + "\n")


In [46]:
def run_branch_coverage_tests():
    save_result("TC_BC_01 Empty list", get_hash([]))
    save_result("TC_BC_02 Non-empty list", get_hash([1]))
    save_result("TC_BC_03 Empty dictionary", get_hash({}))
    save_result("TC_BC_04-A Dict order a→b", get_hash({"a": 1, "b": 2}))
    save_result("TC_BC_04-B Dict order b→a", get_hash({"b": 2, "a": 1}))
    save_result("TC_BC_05-A Nested dict x→y", get_hash({"outer": {"x": 1, "y": 2}}))
    save_result("TC_BC_05-B Nested dict y→x", get_hash({"outer": {"y": 2, "x": 1}}))

class MyClass:
    def __reduce__(self):
        return (MyClass, ())

save_result("TC_BC_06 Custom class object", get_hash(MyClass()))

save_result("TC_BC_07 Float list", get_hash([0.1, 0.2, 0.3]))

In [47]:
run_branch_coverage_tests()

In [1]:
def print_all_result_blocks(file_path="branch_hashes.txt"):
    if not os.path.exists(file_path):
        print("❌ The result file does not exist.")
        return

    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read().strip()

    if content:
        print("✅ All recorded test results:\n")
        print(content)
    else:
        print("⚠️ The result file is empty.")

### 4️⃣ Platform-specific Hash Results
> Hash outputs for each test case executed on macOS, Windows, and Linux.

In [3]:
print_all_result_blocks("branch_hashes.txt")

✅ All recorded test results:

TC_BC_06 Custom class object
Linux     Result: fa18b43d8fea0d6b2fdd5b5939462d340af356328addad61d680f72003ed6e92
macOS     Result: fa18b43d8fea0d6b2fdd5b5939462d340af356328addad61d680f72003ed6e92
Windows   Result: fa18b43d8fea0d6b2fdd5b5939462d340af356328addad61d680f72003ed6e92

TC_BC_07 Float list
Linux     Result: a9af647ab7c3a6dccab3c10e82b0aa990c08880567f392ad82c19583718ba1bb
macOS     Result: a9af647ab7c3a6dccab3c10e82b0aa990c08880567f392ad82c19583718ba1bb
Windows   Result: a9af647ab7c3a6dccab3c10e82b0aa990c08880567f392ad82c19583718ba1bb

TC_BC_01 Empty list
Linux     Result: ec0a6ccf9debf1c16781445c4b9106080d00478b0559469336db7c7b7b9711c8
macOS     Result: ec0a6ccf9debf1c16781445c4b9106080d00478b0559469336db7c7b7b9711c8
Windows   Result: ec0a6ccf9debf1c16781445c4b9106080d00478b0559469336db7c7b7b9711c8

TC_BC_02 Non-empty list
Linux     Result: 4161bccd029bec6dfae5cc2e6620fc7864b70ff481c3a1117f9926854a9b1974
macOS     Result: 4161bccd029bec6dfae5cc2e66

### 5️⃣ Consistency Analysis and Divergence Detection

All test cases were executed on macOS, Windows, and Linux under Python 3.12.4. The serialized outputs from `pickle.dumps()` were hashed using SHA256 and compared across platforms.

**Result:** All hashes matched across all platforms for every test case.

#### ✅ Test cases with consistent hashes on all platforms:

| Test Case ID     | Description                            |
|------------------|----------------------------------------|
| TC_BC_01         | Empty list                             |
| TC_BC_02         | Non-empty list                         |
| TC_BC_03         | Empty dictionary                       |
| TC_BC_04-A       | Dict with keys ordered a → b           |
| TC_BC_04-B       | Dict with keys ordered b → a           |
| TC_BC_05-A       | Nested dict with keys x → y            |
| TC_BC_05-B       | Nested dict with keys y → x            |
| TC_BC_06         | Custom class object with `__reduce__`  |
| TC_BC_07         | Floating-point list                    |

<br>

> Even for inputs that were structurally different but semantically equivalent (e.g., reordered dict keys), the hash results remained consistent across platforms.

### 6️⃣ Conclusions and Findings

The `pickle` module, when tested on macOS, Windows, and Linux (all using Python 3.12.4), showed **fully deterministic behavior** for all input types tested.

#### 🔍 Key Findings:

- ✅ No divergence was observed even with reordered keys in dicts;
- ✅ Python 3.12 likely maintains insertion order or applies stable key handling in `pickle` serialization;
- ✅ Custom objects and floating-point values exhibited identical hashes across platforms.

#### ⚠️ Limitations:

- Only Python 3.12.4 was tested; earlier versions such as 3.7.x or 3.6 may behave differently;
- Input types were basic; deep recursion and complex objects were not included;
- All platforms were assumed to be 64-bit; architecture variation (e.g., ARM vs x86) was not tested.

> Future work should extend to older Python versions, fuzzed/randomized inputs, and testing against different pickle protocols for full consistency guarantees.

### 📎 Appendix: Raw Data File

The complete platform hash records can be found in the following file:

👉 [Download branch_hashes.txt](./branch_hashes.txt)